In [1]:
import azureml

from azureml.core import Run
from azureml.core import Workspace

from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.environment import Environment

from azureml.core.experiment import Experiment

from azureml.core.image import Image

from azureml.core.model import InferenceConfig
from azureml.core.model import Model

from azureml.core.run import Run

from azureml.core.webservice import AciWebservice, Webservice

In [12]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = ""

#Provide values for the existing Resource Group 
resource_group = ""

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = ""
workspace_region = "East US"

model_name = "batt-cycles-7"

description = 'Forecast'
cpu_cores = 1
memory_gb = 1
tags = {'name':'scoring'}

conda_packages = ['numpy', 'pandas', 'scikit-learn', 'py-xgboost<=0.80']
pip_packages = ['azureml-train-automl==1.0.60', 'inference-schema']
conda_file = "dependencies.yml"

env_name = "myenv"
scoring_file = "scoring_service.py"

service_name = "aciservice"

In [3]:
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

In [4]:
scoring_service = """
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

# from inference_schema.schema_decorators import input_schema, output_schema
# from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
# from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

input_sample = pd.DataFrame({'battery_Age_Days': 120, 'daily_Trip_Duration': 590})
### input_sample = pd.DataFrame(data=[{"Date":"2013-01-01T00:00:00.000Z","Battery_ID":0,"Battery_Age_Days":0,"Daily_Trip_Duration":67.8456075842}])

def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = '""" + model_name + """')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
def run(data):
    try:
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

    return json.dumps({"result": result.tolist()})
"""

with open(scoring_file, "w") as file:
  file.write(scoring_service)

In [5]:
registeredModel = Model(workspace = ws, name = model_name)

registeredModel

In [6]:
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = cpu_cores, 
    memory_gb = memory_gb, 
    tags = tags, 
    description = description)

In [8]:
conda_env = CondaDependencies.create(conda_packages = conda_packages, pip_packages = pip_packages)

with open(conda_file, "w") as f:
    f.write(conda_env.serialize_to_string())

In [9]:
myenv = Environment.from_conda_specification(name = env_name, file_path = conda_file)

myenv.register(workspace = ws)

In [10]:
inference_config = InferenceConfig(entry_script = scoring_file, environment = myenv)

In [11]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [16]:
service = Model.deploy(ws, service_name, [registeredModel], inference_config, deployment_config, overwrite = True)

In [ ]:
service.wait_for_deployment(show_output = True)

print(service.state)